In [393]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [402]:
kernel_size = np.ones((3,3))

def image_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

def image_bin(image_gs):
    height, width = image_gs.shape[0:2]
    image_binary = np.ndarray((height, width), dtype=np.uint8)
    ret, image_bin = cv2.threshold(image_gs, 28, 255, cv2.THRESH_BINARY)
    return image_bin

def adaptive_treshold(image):
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,21,4)

def display_image(image, color=False):
    plt.figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')
    if color:
        plt.imshow(image)
    else:
        plt.imshow(image,'gray')
        
def invert(image):
    return 255-image

def dilate(image, iterations, kernel=kernel_size):
    return cv2.dilate(image, kernel, iterations=iterations)

def erode(image, iterations, kernel=kernel_size):
    return cv2.erode(image, kernel, iterations=iterations)

def closing(image, dilate_iterations, erode_iterations, dilate_kernel=kernel_size, erode_kernel=kernel_size):
    return erode(dilate(image, dilate_iterations, dilate_kernel), erode_iterations, erode_kernel)

def opening(image, dilate_iterations, erode_iterations, dilate_kernel=kernel_size, erode_kernel=kernel_size):
    return dilate(erode(image, erode_iterations, erode_kernel), dilate_iterations, dilate_kernel)

def crop_image(image, from_x, to_x, from_y, to_y):
    return image[from_x:to_x, from_y:to_y]

In [403]:
def detect_cross(y, yy):
    return abs(yy -y ) < 7.5

In [406]:
def process_video(video_path):
    
    number_of_people = 0
    
    # crossing line y cordinate
    line_y = 250
    
    # read video
    cap = cv2.VideoCapture(video_path)

    ret_val, frame = cap.read()

    frame_gray = image_gray(frame)

    current_frame = frame
    
    counter = -1
    
    while True:
        success, next_frame = cap.read()
        
        if not success:
            break
            
        counter += 1
        
        # process every fifth frame
        if counter != 5:
            continue
        
        counter = 0
        
        current_frame_gray = image_gray(current_frame)
        next_frame_gray = image_gray(next_frame)
        
        current_frame = next_frame
        
        # difference between current and previous frame
        diff = cv2.absdiff(current_frame_gray, next_frame_gray)

        diff_bin = adaptive_treshold(diff)
        
        diff_bin = closing(diff_bin, 1, 3)
        
        contours, _ = cv2.findContours(diff_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            (x,y), (width, height), _ = cv2.minAreaRect(contour)

            if not 10 < width < 70 or not 10 < height < 70:
                continue
                
            if detect_cross(y, line_y):
                number_of_people += 1
    cap.release()
    return number_of_people

In [407]:
 videos = ["video1.mp4", "video2.mp4", "video3.mp4", "video4.mp4", "video5.mp4", "video6.mp4",
           "video7.mp4", "video8.mp4", "video9.mp4", "video10.mp4"]

number_of_people = [4, 24, 17, 23, 17, 27, 29, 22, 10, 23]
err_sum = 0

for i, video in enumerate(videos):
    people_counted = process_video("data/" + video)
    print ("Number of people: ", people_counted)
    err = abs(number_of_people[i] - people_counted)
    err_sum += err
    
print ("\nMAE: ", err_sum / len(videos))


Number of people:  4
Number of people:  29
Number of people:  18
Number of people:  28
Number of people:  15
Number of people:  30
Number of people:  27
Number of people:  21
Number of people:  9
Number of people:  15

MAE:  2.8
